In [3]:
import pandas as pd
import os
import re
import pytz
from datetime import datetime, timedelta
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.metrics import Recall
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv1D, MaxPooling1D
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow import keras

In [4]:
df_info = pd.read_csv('lista_ico_2020-08-17_tab3.csv', sep=',')

In [5]:
df_info.head()

,ico,adress,fraud,start_date,market_start_date,diff_days,market_days,size_ok,price,market_cap,o_concur,biggest_holder,exchange,contract,date_analysis,code,site,social_media
0,0X,0xe41d2489571d322189246dafa5ebde1f4699f498,0,2017-08-11,2017-08-15,4,1124,NaN,NaN,45.0,x,2,6,14,2018-02-11,NaN,NaN,NaN
1,0XCERT,0x83e2be8d114f9661221384b3a50d24b96a5653f5,1,2018-06-26,2018-07-11,15,794,x,NaN,NaN,NaN,12,2,0,2019-01-07,x,x,x
2,4NEW,0x241ba672574a78a3a604cdd0a94429a73a84a324,1,2018-08-20,2018-08-20,0,754,x,NaN,NaN,NaN,50,7,0,2019-02-16,x,NaN,NaN
3,ABULABA,0xd938137e6d96c72e4a6085412ada2dad78ff89c4,1,2018-04-19,2018-12-16,241,636,NaN,x,NaN,NaN,13,7,41,2019-06-14,x,NaN,NaN
4,AELF,0xbf2179859fc6d5bee9bf9158632dc51678a4100e,0,2017-12-18,2017-12-21,3,996,NaN,NaN,154.0,x,13,10,0,2018-06-19,NaN,NaN,NaN


In [6]:
with open('dict_arrays_length20.pickle','rb') as pickle_file:
    dict_ico_len_20 = pickle.load(pickle_file) 

In [7]:
dict_ico_len_20.get('arrays')

('array_daily_transactions',
 'array_perc_new_holders',
 'array_biggest_holder',
 'array_newbiers',
 'array_gas_ratio',
 'array_daily_transactions_ma',
 'array_perc_new_holders_ma',
 'array_biggest_holder_ma',
 'array_newbiers_ma',
 'array_gas_ratio_ma')

In [8]:
dict_ico_len_20.pop('arrays')

('array_daily_transactions',
 'array_perc_new_holders',
 'array_biggest_holder',
 'array_newbiers',
 'array_gas_ratio',
 'array_daily_transactions_ma',
 'array_perc_new_holders_ma',
 'array_biggest_holder_ma',
 'array_newbiers_ma',
 'array_gas_ratio_ma')

In [9]:
dict_perc_transactions = {k:v[0] for k,v in dict_ico_len_20.items() if len(v[0]) == 20}
dict_perc_new_holders = {k:v[1] for k,v in dict_ico_len_20.items() if len(v[1]) == 20}
dict_perc_big_holder = {k:v[2] for k,v in dict_ico_len_20.items() if len(v[2]) == 20}
dict_perc_newbiers = {k:v[3] for k,v in dict_ico_len_20.items() if len(v[3]) == 20}
dict_gas_ratio = {k:v[4] for k,v in dict_ico_len_20.items() if len(v[4]) == 20}
dict_perc_transactions_ma = {k:v[5] for k,v in dict_ico_len_20.items() if len(v[5]) == 20}
dict_perc_new_holders_ma = {k:v[6] for k,v in dict_ico_len_20.items() if len(v[6]) == 20}
dict_perc_big_holder_ma = {k:v[7] for k,v in dict_ico_len_20.items() if len(v[7]) == 20}
dict_perc_newbiers_ma = {k:v[8] for k,v in dict_ico_len_20.items() if len(v[8]) == 20}
dict_gas_ratio_ma = {k:v[9] for k,v in dict_ico_len_20.items() if len(v[9]) == 20}

In [10]:
df_transactions = pd.DataFrame(dict_perc_transactions).T
df_new_holders = pd.DataFrame(dict_perc_new_holders).T
df_big_holder = pd.DataFrame(dict_perc_big_holder).T
df_newbiers = pd.DataFrame(dict_perc_newbiers).T
df_gasratio = pd.DataFrame(dict_gas_ratio).T
df_transactions_ma = pd.DataFrame(dict_perc_transactions_ma).T
df_new_holders_ma = pd.DataFrame(dict_perc_new_holders_ma).T
df_big_holder_ma = pd.DataFrame(dict_perc_big_holder_ma).T
df_newbiers_ma = pd.DataFrame(dict_perc_newbiers_ma).T
df_gasratio_ma = pd.DataFrame(dict_gas_ratio_ma).T

In [11]:
df_training_transactions = df_transactions.merge(df_info[['ico','fraud']], left_index=True, right_on='ico')
df_training_transactions.set_index('ico', inplace=True)

df_training_new_holders = df_new_holders.merge(df_info[['ico','fraud']], left_index=True, right_on='ico')
df_training_new_holders.set_index('ico', inplace=True)

df_training_big_holder = df_big_holder.merge(df_info[['ico','fraud']], left_index=True, right_on='ico')
df_training_big_holder.set_index('ico', inplace=True)

df_training_newbiers = df_newbiers.merge(df_info[['ico','fraud']], left_index=True, right_on='ico')
df_training_newbiers.set_index('ico', inplace=True)

df_training_gasratio = df_gasratio.merge(df_info[['ico','fraud']], left_index=True, right_on='ico')
df_training_gasratio.set_index('ico', inplace=True)

In [12]:
df_training_transactions_ma = df_transactions_ma.merge(df_info[['ico','fraud']], left_index=True, right_on='ico')
df_training_transactions_ma.set_index('ico', inplace=True)

df_training_new_holders_ma = df_new_holders_ma.merge(df_info[['ico','fraud']], left_index=True, right_on='ico')
df_training_new_holders_ma.set_index('ico', inplace=True)

df_training_big_holder_ma = df_big_holder_ma.merge(df_info[['ico','fraud']], left_index=True, right_on='ico')
df_training_big_holder_ma.set_index('ico', inplace=True)

df_training_newbiers_ma = df_newbiers_ma.merge(df_info[['ico','fraud']], left_index=True, right_on='ico')
df_training_newbiers_ma.set_index('ico', inplace=True)

df_training_gasratio_ma = df_gasratio_ma.merge(df_info[['ico','fraud']], left_index=True, right_on='ico')
df_training_gasratio_ma.set_index('ico', inplace=True)

In [13]:
from ico_parser import ICODeepTraining

In [14]:
model_mlp = Sequential()
model_mlp.add(Dense(20, input_dim=20, activation='relu'))
model_mlp.add(Dense(30, activation='relu'))
model_mlp.add(Dense(20, activation='relu'))
model_mlp.add(Dense(10, activation='relu'))
model_mlp.add(Dense(1, activation='sigmoid'))

In [15]:
ico_training = ICODeepTraining(df_training_transactions.iloc[:, :-1],
                               df_training_transactions.iloc[:, -1],
                               model_mlp,
                               ann_type='mlp',
                               size_array=20)
                               
ico_training.split_train_test()
ico_training.model_summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 20)                420       
_________________________________________________________________
dense_1 (Dense)              (None, 30)                630       
_________________________________________________________________
dense_2 (Dense)              (None, 20)                620       
_________________________________________________________________
dense_3 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 11        
Total params: 1,891
Trainable params: 1,891
Non-trainable params: 0
_________________________________________________________________


In [16]:
ico_training.train_network(loss='binary_crossentropy', 
                      optimizer='adam',
                      metrics=['recall'] ,
                      epochs=100,
                      verbose=0, 
                      batch_size=32,
                      callback=earlystop)

ico_training.plot_training()

NameError: name 'earlystop' is not defined